# Comparison of interventions

In this notebook we compare some of the intervention methods previously introduced with each other.

In [58]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing.eq_odds_postprocessing import (
    EqOddsPostprocessing,
)
from helpers.fairness_measures import accuracy, equalised_odds_d

In [59]:
from helpers import export_plot

## Load data

In [60]:
artifacts_dir = Path("../../artifacts")

In [61]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../artifacts")

In [177]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

In [178]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("sex")

In [106]:
privileged_groups = [{"sex": 1.0}]
unprivileged_groups = [{"sex": 0.0}]

## a) Along fairness-accuracy trade-off

In [65]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from aif360.datasets import StandardDataset
from helpers.fairness_measures import accuracy, equalised_odds_p

### Load original model

In [179]:
bl_model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator LabelBinarizer from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator MLPClassifier from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.



In [180]:
bl_val_pred = bl_model.predict(val.drop("salary", axis=1))
val_sds_pred = val_sds.copy(deepcopy=True)
val_sds_pred.labels = bl_val_pred.reshape(-1, 1)

bl_test_pred = bl_model.predict(test.drop("salary", axis=1))
test_sds_pred = test_sds.copy(deepcopy=True)
test_sds_pred.labels = bl_test_pred.reshape(-1, 1)

## Equalised odds: Post-processing methods

### Hardt et al.

Hardt allows no further parameters that regulate the accuracy-fairness trade-off

In [68]:
from aif360.algorithms.postprocessing.eq_odds_postprocessing import (
    EqOddsPostprocessing,
)

In [69]:
# Learn parameters to equalize odds and apply to create a new dataset
eopp = EqOddsPostprocessing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    seed=np.random.seed(),
)
eopp = eopp.fit(val_sds, val_sds_pred)

In [148]:
test_sds_pred_tranf = eopp.predict(test_sds_pred)
test_sds_pred_tranf.scores = test_sds_pred_tranf.labels

In [149]:
hardt_test_labels = test_sds_pred_tranf.scores.flatten().copy()

In [150]:
hardt_results = {}
hardt_results['eo'] = equalised_odds_d(hardt_test_labels, test.sex, test.salary)
hardt_results['accuracy'] = accuracy(hardt_test_labels, test.salary)

In [151]:
accuracy(hardt_test_labels, test.salary)

0.8239043824701195

### Kamiran et al.

In [109]:
from aif360.algorithms.postprocessing.reject_option_classification import (
    RejectOptionClassification,
)

In [158]:
# Metric used
metric_name = "Average odds difference"  # alias for equalised odds

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

In [181]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

In [182]:
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

In [183]:
kamiran_test_labels = test_pred_labels

In [184]:
kamiran_results = {}
kamiran_results['eo'] = equalised_odds_d(kamiran_test_labels, test.sex.values.flatten(), test.salary.values.flatten())
kamiran_results['accuracy'] = accuracy(kamiran_test_labels, test.salary.values.flatten())

In [185]:
kamiran_test_labels

array([1., 0., 1., ..., 1., 0., 0.])

In [186]:
test.salary.values.flatten()

array([1, 0, 1, ..., 1, 0, 0])

## Visualise outcomes

In [187]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=[hardt_results['eo']], y=[hardt_results['accuracy']], name='Hardt'))
fig.add_trace(go.Scatter(x=[kamiran_results['eo']], y=[kamiran_results['accuracy']], name='Kamiran'))

fig['layout']['yaxis'].update(title='', range=[0, 1.05], dtick=5, autorange=False)
fig['layout']['xaxis'].update(title='', range=[-.05, 0.1], dtick=5, autorange=False)

fig.show()